# Importación de librerias

In [ ]:
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.2 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as px

import PIL.Image

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Creación de variables y fuciones

## Ajustes NST

In [ ]:
STYLE_WEIGHT = 100
CONTENT_WEIGHT = 0.5
Nepochs = 20000
muestra_cada = 200
Imsize = 286

In [ ]:
def vgg_layers(layer_names):
    """ Creates a vgg model that returns a list of intermediate output values."""
    # Load our model. Load pretrained VGG, trained on imagenet data
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False

    outputs = [vgg.get_layer(name).output for name in layer_names]

    model = tf.keras.Model([vgg.input], outputs)
    return model
  

def gram_matrix(input_tensor):
    result = tf.math.reduce_mean(input_tensor, [1,2])
    #input_shape = tf.shape(input_tensor)
    #num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result#/(num_locations)

# Extraer estilo y contenido
class StyleContentModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super(StyleContentModel, self).__init__()
        self.vgg = vgg_layers(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg.trainable = False

    def call(self, inputs):
        "Expects float input in [0,1]"
        inputs = inputs*255.0
        preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed_input)
        style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                          outputs[self.num_style_layers:])

        style_outputs = [gram_matrix(style_output)
                         for style_output in style_outputs]

        content_dict = {content_name: value
                        for content_name, value
                        in zip(self.content_layers, content_outputs)}

        style_dict = {style_name: value
                      for style_name, value
                      in zip(self.style_layers, style_outputs)}

        return {'content': content_dict, 'style': style_dict}


# Función de pérdidas
def style_content_loss(outputs,style_weight,content_weight):
    style_outputs = outputs['style']
    content_outputs = outputs['content']

    style_loss = [style_weight*tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()]

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= content_weight 
    loss = style_loss + content_loss
    
    return loss, style_loss, content_loss


def clip_0_1(image):
      return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

In [ ]:
opt = tf.optimizers.Adam(learning_rate=0.001, beta_1=0.99, epsilon=1e-1)

style_layers_ori = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

content_layers = ['block5_conv2'] 

## Ajustes visualización

In [ ]:
def one_hot_mask(y):
    ''' Do the one hot encoding for the masks.
  
    Arguments:
        - y (tf tensor): Mask of shape (height, width, 3)

    Returns:
        - mask (tf tensor): Mask after do the one hot. Shape (height, width, num_classes) '''

    one_hot_map = []
    for color in colors:
        class_map = tf.reduce_all(tf.equal(y, color), axis = -1)
        one_hot_map.append(class_map)
    mask = tf.cast(tf.stack(one_hot_map, axis = -1), tf.int32)
    return mask


def load_image(folder, file, height = 96, width = 256, crop = False):
    ''' Load and preprocess a train image by:
        - Crop the image to not have the Mercedes-Benz star
        - Resize the image to (height, width)
        - Normalize the image to [0, 1]
  
    Arguments:
        - folder (string): Path to the folder
        - file (string): Name of the file to load
        - height (int): Height to resize -- 96
        - width (int): Width to resize -- 256
        - crop (bool): Crpo the image or not -- True

    Returns:
        - image (tf tensor): Preprocessed image '''

    # Load the image (png)
    image = tf.io.read_file(folder + '/' + file)
    image = tf.cast(tf.image.decode_png(image, channels = 3), tf.float32)

    # Crop the image
    if crop:
        image = tf.image.crop_to_bounding_box(image, 0, 0, 768, 2048)

    # Resize the image
    image = tf.image.resize(image, (height, width))

    # Normalize the image
    image = tf.cast(image, tf.float32)/255.0
    return image


def load_mask(folder, file, height = 96, width = 256, one_hot = True, crop = False):
    ''' Load and preprocess a train mask by:
        - Crop the image to not have the Mercedes-Benz star
        - Resize the image to (height, width)
        - Reshaping the mask from (height, width, 3) to (height, width, 30): One hot encoding
  
    Arguments:
        - folder (string): Path to the folder
        - file (string): Name of the file to load
        - height (int): Height to resize -- 96
        - width (int): Width to resize -- 256
        - one_hot (bool): Do one hot encoding or not -- True
        - crop (bool): Crpo the image or not -- True

    Returns:
        - image (tf tensor): Preprocessed mask '''

    # Load the mask (png)
    image = tf.io.read_file(folder + '/' + file)
    image = tf.cast(tf.image.decode_png(image, channels = 3), tf.int32)

    # Crop the mask
    if crop:
        image = tf.image.crop_to_bounding_box(image, 0, 0, 768, 2048)

    # Resize the mask
    image = tf.image.resize(image, (height, width), method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # One hot encoding
    if one_hot:
        image = one_hot_mask(image)
    return image


def one_hot_to_color_mask(mask, colors, img_height = 96, img_width = 256):
    ''' Convert from the mask from the classes with highest probablities to the correct color. From (96, 256, 1) to (96, 256, 3).
  
    Arguments:
        - mask (tf tensor): Mask with the classes with highest probabilities
        - colors (list): List with the class colors
        - img_height (int): Height of the images -- 96
        - img_width (int): Width of the images -- 256

    Returns:
        - color_mask (tf tensor): Color mask '''

    color_mask = np.zeros((img_height, img_width, channels)).astype('float')
    for c in range(len(colors)):
        color_true = mask == c
        for i in range(3):
            color_mask[:,:,i] += color_true*colors[c][i]

    color_mask = tf.cast(color_mask, dtype = tf.int32)

    return color_mask

def load_train(image_name, mask_name):
    ''' Load and preprocess a train image and its mask
  
    Arguments:
        - image_name (string): Name of the image to load
        - mask_name (string): Name of the mask to load

    Returns:
        - image (tf tensor): Preprocessed image
        - mask (tf tensor): Preprocessed mask '''

    image = load_image(train_images_folder_path, image_name, img_height, img_width)
    mask = load_mask(train_mask_folder_path, mask_name, img_height, img_width)
    return image, mask

In [ ]:
train_images_folder_path = "/content/drive/My Drive" 
train_mask_folder_path = "/content/drive/MyDrive/GTA Segmentacion" 

img_height, img_width, channels = 96, 256, 3

batch_size = 2

colors = np.array([(0, 0, 0), (111, 74, 0), (81, 0, 81), (128, 64, 128), (244, 35, 232), (250, 170, 160), (230, 150, 140), (70, 70, 70), (102, 102, 156), (190, 153, 153), (180, 165, 180), 
                   (150, 100, 100), (150, 120, 90), (153, 153, 153), (250, 170, 30), (220, 220, 0), (107, 142, 35), (152, 251, 152), (70, 130, 180), (220, 20, 60), (255, 0, 0), ( 0, 0, 142), 
                   ( 0, 0, 70), (0, 60, 100), (0, 0, 90), (0, 0, 110), (0, 80, 100), (0, 0, 230), (119, 11, 32), (0, 0, 142)], dtype = np.int32)

model = tf.keras.models.load_model('/content/drive/My Drive/Modelo_segmentacion_UNET/best_model_weights_and_architecture')

# Inicialización

In [ ]:
path_result = "/content/drive/MyDrive/Resultados/"

path_GTA = "/content/drive/MyDrive/GTA/"
path_Reales = "/content/drive/MyDrive/Reales/"

itera = 60  # Número de imagenes de prueba

list_GTA = os.listdir(path_GTA)
images_GTA = random.choices(list_GTA, k=itera)

list_Reales = os.listdir(path_Reales)
images_Reales = random.choices(list_Reales, k=itera)

for i in range(itera):
  images_GTA[i] = path_GTA + images_GTA[i]
  images_Reales[i] = path_Reales + images_Reales[i]

## Cambiar alguna imagen si hace falta

In [ ]:
a = ["01526.png", "00031.png", "00513.png", "01166.png", "01151.png", "01782.png", "01550.png", "01450.png", "00557.png", "02078.png",  # 10 primeros
     "01575.png", "00370.png", "01401.png", "01999.png", "00203.png", "00560.png", "00142.png", "02074.png", "02406.png", "00467.png",
     "01889.png", "00466.png", "02307.png", "00529.png", "02071.png", "00045.png", "00301.png", "00491.png", "00078.png", "01289.png",
     "01862.png", "02264.png", "01297.png", "01385.png", "01348.png", "02019.png", "02099.png", "00242.png", "01101.png", "02313.png",
     "01655.png", "00923.png", "01495.png", "01947.png", "02404.png", "01293.png", "00733.png", "01579.png", "01701.png", "00248.png", 
     "01597.png", "01665.png", "01909.png", "00857.png", "00646.png", "02134.png", "00993.png", "01394.png", "01600.png", "01012.png"]

b = ["bremen_000152_000019_leftImg8bit.png", "stuttgart_000115_000019_leftImg8bit.png", "dusseldorf_000141_000019_leftImg8bit.png",
      "monchengladbach_000000_007098_leftImg8bit.png", "cologne_000006_000019_leftImg8bit.png", "bremen_000256_000019_leftImg8bit.png",
      "bremen_000190_000019_leftImg8bit.png", "cologne_000072_000019_leftImg8bit.png", "cologne_000064_000019_leftImg8bit.png", "strasbourg_000000_029020_leftImg8bit.png", # 10 primeros
      "aachen_000137_000019_leftImg8bit.png", "dusseldorf_000030_000019_leftImg8bit.png", "bremen_000004_000019_leftImg8bit.png", "bremen_000085_000019_leftImg8bit.png",
      "dusseldorf_000026_000019_leftImg8bit.png", "bochum_000000_009951_leftImg8bit.png", "tubingen_000120_000019_leftImg8bit.png", "hamburg_000000_032906_leftImg8bit.png",
      "hanover_000000_055800_leftImg8bit.png", "monchengladbach_000000_033454_leftImg8bit.png", "stuttgart_000065_000019_leftImg8bit.png", "bremen_000183_000019_leftImg8bit.png",
      "bremen_000079_000019_leftImg8bit.png", "stuttgart_000087_000019_leftImg8bit.png", "tubingen_000035_000019_leftImg8bit.png", "strasbourg_000001_022151_leftImg8bit.png",
      "aachen_000140_000019_leftImg8bit.png", "cologne_000094_000019_leftImg8bit.png", "jena_000052_000019_leftImg8bit.png", "tubingen_000059_000019_leftImg8bit.png",
      "aachen_000009_000019_leftImg8bit.png", "darmstadt_000074_000019_leftImg8bit.png", "strasbourg_000000_013863_leftImg8bit.png", "aachen_000097_000019_leftImg8bit.png",
      "dusseldorf_000033_000019_leftImg8bit.png", "cologne_000145_000019_leftImg8bit.png", "krefeld_000000_021222_leftImg8bit.png", "hamburg_000000_078407_leftImg8bit.png",
      "hamburg_000000_053486_leftImg8bit.png", "hanover_000000_026804_leftImg8bit.png", "bremen_000121_000019_leftImg8bit.png", "stuttgart_000012_000019_leftImg8bit.png",
      "bremen_000135_000019_leftImg8bit.png", "jena_000070_000019_leftImg8bit.png", "bochum_000000_023040_leftImg8bit.png", "jena_000108_000019_leftImg8bit.png",
      "stuttgart_000155_000019_leftImg8bit.png", "bochum_000000_030913_leftImg8bit.png", "strasbourg_000001_031272_leftImg8bit.png", "tubingen_000075_000019_leftImg8bit.png",
      "tubingen_000048_000019_leftImg8bit.png", "cologne_000005_000019_leftImg8bit.png", "dusseldorf_000097_000019_leftImg8bit.png", "bremen_000059_000019_leftImg8bit.png",
      "hamburg_000000_098400_leftImg8bit.png", "hamburg_000000_081299_leftImg8bit.png", "aachen_000004_000019_leftImg8bit.png", "hamburg_000000_079376_leftImg8bit.png",
      "bremen_000112_000019_leftImg8bit.png", "monchengladbach_000000_006518_leftImg8bit.png"]

for i in range(itera):
  images_GTA[i] = path_GTA + a[i]
  images_Reales[i] = path_Reales + b[i]

## Para saber que imágenes que se han escogido

In [ ]:
for name in images_GTA:
  print(name.split("/")[-1])
print("------------------------------------------------")
for name in images_Reales:
  print(name.split("/")[-1])

# Automático para muchos

In [ ]:
n_style = [50,75,100,200,500,1000]

for num in range(0, itera):

# NST  &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
  style_image = PIL.Image.open(images_Reales[num])
  content_image = PIL.Image.open(images_GTA[num])

  content_image = np.asarray(content_image.resize((Imsize,Imsize)))
  style_image = np.asarray(style_image.resize((Imsize,Imsize)))

  content_image = content_image[:,:,0:3].astype(np.float32)/255
  style_image = style_image[:,:,0:3].astype(np.float32)/255

  content_image = np.expand_dims(content_image,0)
  style_image = np.expand_dims(style_image,0)


  for s in n_style:
    STYLE_WEIGHT = s # Cambiamos la cantidad de estilo en cada irteración

    for layer in np.arange(1,2):
      style_layers = style_layers_ori[0:5]

      extractor = StyleContentModel(style_layers, content_layers)
      style_targets = extractor(style_image)['style']
      content_targets = extractor(content_image)['content']

      image = tf.Variable(content_image.copy())

      style_weight_FG=5*1e-11
      content_weight_FG=1e-4

      loss, style_loss, content_loss = style_content_loss(extractor(content_image),style_weight_FG,content_weight_FG)
      loss_style = np.array(style_loss[0])

      loss, style_loss, content_loss = style_content_loss(extractor(style_image),style_weight_FG,content_weight_FG)
      loss_content = np.array(content_loss)

      style_weight= style_weight_FG * STYLE_WEIGHT/loss_style
      content_weight= content_weight_FG * CONTENT_WEIGHT/loss_content

      loss, style_loss, content_loss = style_content_loss(extractor(content_image),style_weight,content_weight)

      loss, style_loss, content_loss = style_content_loss(extractor(style_image),style_weight,content_weight)

      @tf.function()
      def train_step(image):
          with tf.GradientTape() as tape:
              outputs = extractor(image)
              loss = style_content_loss(outputs,style_weight,content_weight)

          grad = tape.gradient(loss[1][0], image)
          opt.apply_gradients([(grad, image)])
          image.assign(clip_0_1(image))

      L_cont = np.array([])
      L_style = np.array([]) 

      for n in range(0,Nepochs):
          train_step(image)
          
          if np.mod(n,muestra_cada)==0:
              outputs = extractor(image)
              loss, style_loss, content_loss = style_content_loss(outputs,style_weight,content_weight)
              
              L_cont = np.append(L_cont,np.array(content_loss))
              L_style = np.append(L_style,np.array(style_loss[0]))

      # Resultados
      plt.figure()
      plt.imshow(image[0, :,:,:],vmin=0,vmax=1), plt.axis('off')
      plt.savefig(path_result + "AA_NST_" + str(s) + "_" + str(num+1) + ".png", bbox_inches="tight", pad_inches=0)


# Segmentación  &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
  images_name = ["GTA/" + images_GTA[num].split("/")[-1], "Resultados/AA_NST_50_" + str(num+1) + ".png", "Resultados/AA_NST_75_" + str(num+1) + ".png", "Resultados/AA_NST_100_" + str(num+1) + ".png"]

  mask_names = ((images_name[0].split("/")[-1] + " ") * 4).split()

  dataset = tf.data.Dataset.from_tensor_slices((images_name, mask_names))
  dataset = dataset.map(load_train, num_parallel_calls = tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)

  predictions = model.predict(dataset, batch_size = 10)
  predictions = np.argmax(predictions, axis = 3)

  idx = range(len(images_name))
  visualize_images = [load_image(train_images_folder_path, images_name[i], img_height, img_width) for i in idx]
  visualize_masks = [load_mask(train_mask_folder_path, mask_names[i], img_height, img_width, one_hot = False) for i in idx]
  preds = predictions[idx]

  iou_score = []
  iou_score_half = []

  plt.figure(figsize = (16, 4))
  plt.subplots_adjust(hspace = 0.1)

  for k in range(4):
    plt.subplot(3, 4, k + 1)
    plt.imshow(visualize_images[k])
    # if k==0:
    #   plt.ylabel("Input")
    # plt.title(["Original", "50", "75", "100"][k])
    plt.yticks(())
    plt.xticks(())

    plt.subplot(3, 4, k + 5)
    plt.imshow(visualize_masks[k])
    # if k==0:
    #   plt.ylabel("Ground Truth")
    plt.yticks(())
    plt.xticks(())

    output = one_hot_to_color_mask(preds[k], colors)
# IoU calculation
    intersection = 0
    union = img_height * img_width

    for i in range(img_height):
      for j in range(img_width):
        a = visualize_masks[k][i-1,j-1,:] == output[i-1,j-1,:]
        if sum(a.numpy())==3:
          intersection += 1

    iou_score.append(round(intersection/union, ndigits=2))

# Calcula de la mitad hacia abajo
    intersection = 0
    union = int(img_height/2) * img_width

    for i in range(int(img_height/2), img_height):
        for j in range(img_width):
          a = visualize_masks[k][i-1,j-1,:] == output[i-1,j-1,:]
          if sum(a.numpy())==3:
            intersection += 1

    iou_score_half.append(round(intersection/union, ndigits=2))

    plt.subplot(3, 4, k + 9)
    plt.imshow(output)
    # if k==0:
    #   plt.ylabel("Output")
    plt.xlabel("IoU: %s" % iou_score[k])
    plt.yticks(())
    plt.xticks(())

  plt.savefig(path_result + "A_NSTs_" + str(num+1) + "_1.png", bbox_inches="tight", pad_inches=0)

# Segmentación 2  &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
  images_name = ["GTA/" + images_GTA[num].split("/")[-1], "Resultados/AA_NST_200_" + str(num+1) + ".png", "Resultados/AA_NST_500_" + str(num+1) + ".png", "Resultados/AA_NST_1000_" + str(num+1) + ".png"]

  dataset = tf.data.Dataset.from_tensor_slices((images_name, mask_names))
  dataset = dataset.map(load_train, num_parallel_calls = tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)

  predictions = model.predict(dataset, batch_size = 10)
  predictions = np.argmax(predictions, axis = 3)

  visualize_images = [load_image(train_images_folder_path, images_name[i], img_height, img_width) for i in idx]
  preds = predictions[idx]

  plt.figure(figsize = (16, 4))
  plt.subplots_adjust(hspace = 0.1)

  for k in range(4):
    plt.subplot(3, 4, k + 1)
    plt.imshow(visualize_images[k])
    # if k==0:
    #   plt.ylabel("Input")
    # plt.title(["Original", "200", "500", "1000"][k])
    plt.yticks(())
    plt.xticks(())

    plt.subplot(3, 4, k + 5)
    plt.imshow(visualize_masks[k])
    # if k==0:
    #   plt.ylabel("Ground Truth")
    plt.yticks(())
    plt.xticks(())

    output = one_hot_to_color_mask(preds[k], colors)
# IoU calculation
    intersection = 0
    union = img_height * img_width

    for i in range(img_height):
      for j in range(img_width):
        a = visualize_masks[k][i-1,j-1,:] == output[i-1,j-1,:]
        if sum(a.numpy())==3:
          intersection += 1

    iou_score.append(round(intersection/union, ndigits=2))
    
# Calcula de la mitad hacia abajo
    intersection = 0
    union = int(img_height/2) * img_width

    for i in range(int(img_height/2), img_height):
        for j in range(img_width):
          a = visualize_masks[k][i-1,j-1,:] == output[i-1,j-1,:]
          if sum(a.numpy())==3:
            intersection += 1

    iou_score_half.append(round(intersection/union, ndigits=2))

    plt.subplot(3, 4, k + 9)
    plt.imshow(output)
    # if k==0:
    #   plt.ylabel("Output")
    plt.xlabel("IoU: %s" % iou_score[k+4])
    plt.yticks(())
    plt.xticks(())

  plt.savefig(path_result + "A_NSTs_" + str(num+1) + "_2.png", bbox_inches="tight", pad_inches=0)

# Visualización final 3 (gráfico) &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
  iou = []
  iou_half = []
  for i in range(len(iou_score)):
    if i != 4:
      iou.append(iou_score[i])
      iou_half.append(iou_score_half[i])

  fig = px.Figure(data=[px.Scatter(name = "Completa", x = [0,50,75,100,200,500,1000], y = iou),
                        px.Scatter(name = "Mitad", x = [0,50,75,100,200,500,1000], y = iou_half)])

  fig.update_layout(title="IoU respecto a la cantidad de estilo",
                    xaxis_title="Cantidad de estilo",
                    yaxis_title="IoU",
                    autosize=False)
  fig.update_yaxes(range=[0, 1.05])
  fig.show()
  fig.write_image(path_result + "A_comparacion_NST_" + str(num+1) + ".png")

  print("Completado el", str(num+1))